In [13]:
import torchvision
import torch.nn as nn
import torch
from torch.utils.data import DataLoader, TensorDataset
from torchvision import transforms
image_path ='./'
transform = transforms.Compose([transforms.ToTensor()])
mnist_train_dataset = torchvision.datasets.MNIST(root=image_path, train=True,transform=transform,download=True)
mnist_test_dataset= torchvision.datasets.MNIST(root=image_path,transform=transform,train=False,download=True)

batch_size = 64
torch.manual_seed(1)
train_dl = DataLoader(mnist_train_dataset, batch_size, shuffle=True)

In [14]:
hidden_units = [32,16]
image_size = mnist_train_dataset[0][0].shape
input_size = image_size[0]*image_size[1]*image_size[2]
all_layers = [nn.Flatten()]
for hidden_unit in hidden_units:
  layer = nn.Linear(input_size,hidden_unit)
  all_layers.append(layer)
  all_layers.append(nn.ReLU())
  input_size = hidden_unit
all_layers.append(nn.Linear(hidden_units[-1],10))
model = nn.Sequential(*all_layers)
model

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=32, bias=True)
  (2): ReLU()
  (3): Linear(in_features=32, out_features=16, bias=True)
  (4): ReLU()
  (5): Linear(in_features=16, out_features=10, bias=True)
)

In [15]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr =0.001)
torch.manual_seed(1)
num_epochs=20
for epoch in range(num_epochs):
  accuracy_hist_train = 0
  for x_batch, y_batch in train_dl:
    pred = model(x_batch)
    loss = loss_fn(pred,y_batch)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    is_correct = (torch.argmax(pred,dim=1)== y_batch).float()
    accuracy_hist_train += is_correct.sum()
  accuracy_hist_train /= len(train_dl.dataset)
  print(f'Epoch {epoch}   Accuracy: {accuracy_hist_train:.4f}')

Epoch 0   Accuracy: 0.8531
Epoch 1   Accuracy: 0.9287
Epoch 2   Accuracy: 0.9413
Epoch 3   Accuracy: 0.9506
Epoch 4   Accuracy: 0.9558
Epoch 5   Accuracy: 0.9592
Epoch 6   Accuracy: 0.9627
Epoch 7   Accuracy: 0.9649
Epoch 8   Accuracy: 0.9673
Epoch 9   Accuracy: 0.9690
Epoch 10   Accuracy: 0.9711
Epoch 11   Accuracy: 0.9729
Epoch 12   Accuracy: 0.9737
Epoch 13   Accuracy: 0.9747
Epoch 14   Accuracy: 0.9766
Epoch 15   Accuracy: 0.9778
Epoch 16   Accuracy: 0.9780
Epoch 17   Accuracy: 0.9798
Epoch 18   Accuracy: 0.9806
Epoch 19   Accuracy: 0.9816


In [16]:
pred = model(mnist_test_dataset.data/255.)
is_correct= (torch.argmax(pred,dim=1)==mnist_test_dataset.targets).float()
print(f'Test Accuracy: {is_correct.mean():.4f}')

Test Accuracy: 0.9645


In [19]:
#pytorch lightning
!pip install pytorch_lightning
!pip install torchmetrics
import pytorch_lightning as pl
import torch
import torch.nn as nn
from torchmetrics.classification.accuracy import Accuracy

In [54]:
class MultiLayerPerceptron(pl.LightningModule):
  def __init__(self,image_shape=(1,28,28), hidden_units=(32,16)):
    super().__init__()

    self.train_acc = Accuracy(task="multiclass",num_classes=10)
    self.valid_acc = Accuracy(task="multiclass",num_classes=10)
    self.test_acc = Accuracy(task="multiclass",num_classes=10)

    input_size = image_shape[0]*image_shape[1]*image_shape[2]
    all_layers = [nn.Flatten()]
    for hidden_unit in hidden_units:
      layer = nn.Linear(input_size,hidden_unit)
      all_layers.append(layer)
      all_layers.append(nn.ReLU())
      input_size = hidden_unit
    all_layers.append(nn.Linear(hidden_units[-1],10))
    self.model = nn.Sequential(*all_layers)

  def forward(self,x):
    x =self.model(x)
    return x

  def training_step(self,batch,batch_idx):
    x, y = batch
    logits = self(x)
    loss = nn.functional.cross_entropy(self(x),y)
    preds = torch.argmax(logits,dim=1)
    self.train_acc.update(preds,y)
    self.log("train loss ", loss, prog_bar = True)
    return loss

  def on_training_epoch_end(self,outs):
    self.log("train acc", self.tain_acc.compute())

  def validation_step(self, batch, batch_idx):
    x, y = batch
    logits = self(x)
    loss = nn.functional.cross_entropy(self(x),y)
    preds = torch.argmax(logits,dim=1)
    self.valid_acc.update(preds,y)
    self.log("valid_loss", loss, prog_bar=True)
    self.log("valid _acc", self.valid_acc.compute(), prog_bar=True)
    return loss

  def test_step(self,batch,batch_idx):
    x,y = batch
    logits = self(x)
    loss = nn.functional.cross_entropy(self(x),y)
    preds = torch.argmax(logits,dim=1)
    self.test_acc.update(preds,y)
    self.log("test_loss", loss,prog_bar=True)
    self.log("test acc", self.test_acc.compute(), prog_bar=True)
    return loss

  def configure_optimizers(self):
    optimizer = torch.optim.Adam(self.parameters(), lr=0.001)
    return optimizer

In [55]:
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torchvision.datasets import MNIST
from torchvision import transforms

In [60]:
class MnistDataModule(pl.LightningDataModule):
  def __init__(self,data_path="./"):
    super().__init__()
    self.data_path = data_path
    self.transform = transforms.Compose([transforms.ToTensor()])

  def prepare_data(self) ->None:
    MNIST(root=self.data_path, download=True)

  def setup(self, stage: None) -> None:
    mnist_all = MNIST(self.data_path, train=True,transform=self.transform, download=True)
    self.train, self.val = random_split(mnist_all, [55000,5000], generator=torch.Generator().manual_seed(1))
    self.test = MNIST(self.data_path, train=False, transform=self.transform, download=True)

  def train_dataloader(self):
    return DataLoader(self.train, batch_size=64, num_workers=4)

  def val_dataloader(self):
    return DataLoader(self.val, batch_size=64, num_workers=4)

  def test_dataloader(self):
    return DataLoader(self.test, batch_size=64,num_workers=4)

In [61]:
torch.manual_seed(1)
mnist_dm = MnistDataModule()

In [64]:
mnistclassifier = MultiLayerPerceptron()
if torch.cuda.is_available():
  trainer = pl.Trainer(max_epochs=10, gpus=2)
else:
  trainer = pl.Trainer(max_epochs=10)
trainer.fit(model=mnistclassifier, datamodule=mnist_dm)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type               | Params
-------------------------------------------------
0 | train_acc | MulticlassAccuracy | 0     
1 | valid_acc | MulticlassAccuracy | 0     
2 | test_acc  | MulticlassAccuracy | 0     
3 | model     | Sequential         | 25.8 K
-------------------------------------------------
25.8 K    Trainable params
0         Non-trainable params
25.8 K    Total params
0.103     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


In [65]:
trainer.test(model=mnistclassifier,datamodule=mnist_dm)

Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test acc          │    0.9552987217903137     │
│         test_loss         │    0.13159459829330444    │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.13159459829330444, 'test acc': 0.9552987217903137}]